In [1]:
def count_sentences(conllu_file_path):
    sentence_count = 0

    with open(conllu_file_path, 'r') as file:
        for line in file:
            if line.strip() == '':
                sentence_count += 1

    return sentence_count


file_path = '/home/preetamray-pg/g2g-transformer/data/UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
num_sentences = count_sentences(file_path)
print(f"Number of sentences: {num_sentences}")

Number of sentences: 1315


In [2]:
import numpy as np

def check_head_conllu(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    count = 0
    for i, line in enumerate(lines):
        # print(line)
        if line.startswith('#'):
            continue  # Skip comment lines
        elif line.strip() == '':
            continue  # Skip empty lines
        else:
            fields = line.split('\t')
            if fields[6] == '_':
                count += 1
                print(i+1)
                # Insert a random integer ranging from 0 to the length of the line
                # random_integer = 0
                # fields[6] = str(random_integer)
                lines[i] = '\t'.join(fields)

    # Update the file with modified lines
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

    return count


In [4]:
check_head_conllu("/home/preetamray-pg/G2GTr/data/gold/train.conllx")

0

In [5]:
def is_root_at_last_or_second_last(conllu_sentence):
    lines = conllu_sentence.split('\n')

    # Iterate through the lines to find the root
    root_position = None
    for line in lines:
        if line and not line.startswith('#'):  # Ignore comments
            columns = line.split('\t')
            if len(columns) >= 7 and columns[6] == '0':  # Check if it's a root
                root_position = int(columns[0])

    # Check if the root is at the last or second-last position
    return root_position == len(lines) - 1 or root_position == len(lines) - 2


False


In [18]:
import random

def read_conllu(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conllu_data = file.read()
    return conllu_data

def write_conllu(file_path, conllu_data):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(conllu_data)

def shuffle_deprel_column(conllu_data):
    # Split the data into lines
    lines = conllu_data.strip().split('\n')

    # Parse each line and shuffle the deprel column
    for i, line in enumerate(lines):
        if not line.startswith('#'):
            columns = line.split('\t')
            if len(columns) > 7:  # Ensure the line has enough columns
                deprel_index = 7  # Assuming deprel is at column 8 (0-indexed)
                deprels = columns[deprel_index].split('|')
                random.shuffle(deprels)
                columns[deprel_index] = '|'.join(deprels)
                lines[i] = '\t'.join(columns)

    # Join the lines back into a single string
    shuffled_conllu_data = '\n'.join(lines)
    return shuffled_conllu_data


if __name__ == '__main__':
    input_file = '/home/preetam_pg/g2g-transformer/data/org/train-pos.conllu'
    output_file = '/home/preetam_pg/g2g-transformer/data/org/train-neg.conllu'

    conllu_data = read_conllu(input_file)
    shuffle_deprel_column(conllu_data)
    write_conllu(output_file, conllu_data)

    print(f"Randomly shuffled 'deps' column and saved to {output_file}.")


Randomly shuffled 'deps' column and saved to /home/preetam_pg/g2g-transformer/data/org/train-neg.conllu.


In [14]:
import random

def shuffle_deprels_on_each_word(conllu_sentence):
    lines = conllu_sentence.strip().split('\n')
    shuffled_lines = []

    for line in lines:
        fields = line.split('\t')
        if len(fields) >= 8:  # Make sure there is a 7th column (deprel)
            deprels = fields[7].split('|')
            shuffled_deprels = random.sample(deprels, len(deprels))
            fields[7] = '|'.join(shuffled_deprels)

        shuffled_lines.append('\t'.join(fields))

    return '\n'.join(shuffled_lines)

def shuffle_deprels_in_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        conllu_data = infile.read()

    shuffled_data = shuffle_deprels_on_each_word(conllu_data)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(shuffled_data)

# Example usage
input_file_path = '/home/preetam_pg/g2g-transformer/data/org/train-pos.conllu'
output_file_path = '/home/preetam_pg/g2g-transformer/data/org/train-neg.conllu'

shuffle_deprel_in_file(input_file_path, output_file_path)


In [26]:
import random

def collect_unique_deprels(conllu_data):
    unique_deprels = set()

    lines = conllu_data.strip().split('\n')
    for line in lines:
        fields = line.split('\t')
        if len(fields) >= 8:  # Make sure there is a 7th column (deprel)
            deprels = fields[7].split('|')
            unique_deprels.update(deprels)

    return list(unique_deprels)

def replace_deprels_with_random(conllu_sentence, unique_deprels):
    lines = conllu_sentence.strip().split('\n')
    replaced_lines = []

    for line in lines:
        fields = line.split('\t')
        if len(fields) >= 8:  # Make sure there is a 7th column (deprel)
            deprels = fields[7].split('|')
            # print(deprels)
            shuffled_deprel = random.choice(unique_deprels)
            fields[7] = '|'.join([shuffled_deprel for _ in deprels])

        replaced_lines.append('\t'.join(fields))

    return '\n'.join(replaced_lines)

def replace_deprels_in_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        conllu_data = infile.read()

    unique_deprels = collect_unique_deprels(conllu_data)
    # print(unique_deprels)
    replaced_data = replace_deprels_with_random(conllu_data, unique_deprels)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(replaced_data)


# Example usage
input_file_path = '/home/preetam_pg/g2g-transformer/data/org/train-pos.conllu'
output_file_path = '/home/preetam_pg/g2g-transformer/data/org/train-neg.conllu'

replace_deprels_in_file(input_file_path, output_file_path)


In [7]:
from conllu import parse, parse_tree
import random

def permute_sentence(sentence):
    # Shuffle the word order
    random.shuffle(sentence)

    # Shuffle the deprel (dependency relations)
    for token in sentence:
        token['deprel'] = random.choice(token['deprel'])

    return sentence

def permute_conllu_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    # Parse the CoNLL-U file
    sentences = parse(data)

    # Permute each sentence in the file
    permuted_sentences = [permute_sentence(sentence) for sentence in sentences]

    # Convert back to CoNLL-U format
    permuted_data = '\n\n'.join('\n'.join(token_list.serialize()) for token_list in permuted_sentences)

    # Save the permuted data to a new file
    with open("/home/preetam_pg/g2g-transformer/data/org/train-pos.conllu", 'w', encoding='utf-8') as file:
        file.write(permuted_data)

# Example usage
permute_conllu_file('/home/preetam_pg/g2g-transformer/data/org/train.conllu')

In [20]:
import random

def read_conllu_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conllu_data = file.read().strip().split('\n\n')
    return conllu_data

def shuffle_sentence(sentence):
    lines = sentence.split('\n')
    random.shuffle(lines)  # Shuffle the lines to shuffle the word order
    shuffled_sentence = '\n'.join(lines)
    return shuffled_sentence

def shuffle_conllu_data(conllu_data):
    shuffled_data = [shuffle_sentence(sentence) for sentence in conllu_data]
    return shuffled_data

def write_conllu_file(file_path, conllu_data):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(conllu_data))

if __name__ == "__main__":
    input_file_path = '/home/preetam_pg/g2g-transformer/data/org/train.conllu'
    output_file_path = '/home/preetam_pg/g2g-transformer/data/org/train-pos.conllu'

    # Step 1: Read the CoNLL-U file
    conllu_data = read_conllu_file(input_file_path)

    # Step 2 and 3: Shuffle word order and relations
    shuffled_data = shuffle_conllu_data(conllu_data)

    # Step 4: Write shuffled data to a new CoNLL-U file
    write_conllu_file(output_file_path, shuffled_data)

    print(f"Shuffled data written to {output_file_path}")


Shuffled data written to /home/preetam_pg/g2g-transformer/data/org/train-pos.conllu


In [19]:
original_sentence = """
1   This    _   DET DT  _   2   det _   _
2   is  _   VERB    VBZ _   0   root    _   _
3   an  _   DET DT  _   4   det _   _
4   example _   NOUN    NN  _   2   attr    _   _
5   sentence    _   NOUN    NN  _   4   nsubj   _   _
6   in  _   ADP IN  _   5   prep    _   _
7   CoNLL-U _   PROPN   NN  _   6   pobj    _   _
8   format  _   NOUN    NN  _   6   appos   _   _
"""

shuffled_sentence = shuffle_conllu(original_sentence)
print(shuffled_sentence)

1
2
3
4
5
6
7
8


In [52]:
import random
import numpy as np

def shuffle_conllu_sentence(sentence):
    lines = sentence.strip().split('\n')
    tokens = [line.split('\t') for line in lines if not line.startswith('#')]
    # print(tokens)
    
    # Extract word indices
    indices = [*range(1, len(tokens)+1, 1)]
    # print(indices)
    
    # Shuffle indices
    ind = [*range(1, len(indices)+1, 1)]
    # print(ind)
    shuffled_indices = random.sample(ind, len(ind))
    # print(indices, shuffled_indices)
    
    # Create a mapping from old indices to shuffled indices
    index_mapping = {old_index: new_index for old_index, new_index in zip(indices, shuffled_indices)}
    print(index_mapping)
    
    # Update the tokens with shuffled indices
    for token in tokens:
        # update the head index
        t = token[0]
        old_index = int(t)
        new_index = index_mapping[old_index]
        token[0] = str(new_index)

        # update the head dependency
        t = token[6]
        # print(t)
        old_index = int(t)
        if old_index != 0:
            new_index = index_mapping[old_index]
            token[6] = str(new_index)
        else:
            continue
    
    # Reconstruct the sentence without sorting tokens
    shuffled_sentence = '\n'.join(['\t'.join(token) for token in tokens])
    
    return shuffled_sentence

# Example usage
conllu_sentence = """
1	aTa	_	ind	_	_	8	sambanxah	_	_
2	saH	_	nomsgm	_	_	8	karwa	_	_
3	anugiram	_	ind	_	_	8	axikaranam	_	_
4	ftuBiH	_	instplm	_	_	5	karwa	_	_
5	vitAyamAnAM	_	pprmd	_	_	6	viseranam	_	_
6	vanaantalakzmIM	_	accsgf	_	_	8	karma	_	_
7	vilokayituM	_	inf	_	_	8	prayojanam	_	_
8	niragamat	_	aorsg3	_	_	0	root	_	_
"""

shuffled_conllu_sentence = shuffle_conllu_sentence(conllu_sentence)
print(shuffled_conllu_sentence) 


{1: 6, 2: 1, 3: 8, 4: 2, 5: 7, 6: 3, 7: 4, 8: 5}
6	aTa	_	ind	_	_	5	sambanxah	_	_
1	saH	_	nomsgm	_	_	5	karwa	_	_
8	anugiram	_	ind	_	_	5	axikaranam	_	_
2	ftuBiH	_	instplm	_	_	7	karwa	_	_
7	vitAyamAnAM	_	pprmd	_	_	3	viseranam	_	_
3	vanaantalakzmIM	_	accsgf	_	_	5	karma	_	_
4	vilokayituM	_	inf	_	_	5	prayojanam	_	_
5	niragamat	_	aorsg3	_	_	0	root	_	_


#### To use for random permutation of Prose to create Poetry

In [1]:
import random

def shuffle_conllu_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        sentences = infile.read().strip().split('\n\n')

    shuffled_sentences = [shuffle_conllu_sentence(sentence) for sentence in sentences]

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write('\n\n'.join(shuffled_sentences))

def shuffle_conllu_sentence(sentence):
    lines = sentence.strip().split('\n')
    tokens = [line.split('\t') for line in lines if not line.startswith('#')]
    
    indices = [*range(1, len(tokens)+1, 1)]
    ind = [*range(1, len(indices)+1, 1)]
    shuffled_indices = random.sample(ind, len(ind))
    
    index_mapping = {old_index: new_index for old_index, new_index in zip(indices, shuffled_indices)}
    
    # tokens.sort(key=lambda x: int(x[0]))

    for token in tokens:
        # Update the head index
        t = token[0]
        if '-' not in t:
            old_index = int(t)
            new_index = index_mapping[old_index]
            token[0] = str(new_index)
        else:
            continue

        # Update the head dependency
        t = token[6]
        old_index = int(t)
        if old_index != 0:
            new_index = index_mapping[old_index]
            token[6] = str(new_index)
        else:
            continue
    
    tokens.sort(key=lambda x: int(x[0]))
    shuffled_sentence = '\n'.join(['\t'.join(token) for token in tokens])
    
    return shuffled_sentence

# Example usage
input_conllu_file = "/home/preetamray-pg/g2g-transformer/data/UD_Turkish-Penn/tr_penn-ud-train.conllu"
output_conllu_file = "/home/preetamray-pg/g2g-transformer/data/UD_Turkish-Penn/tr_penn-ud-train-pos.conllu"
shuffle_conllu_file(input_conllu_file, output_conllu_file)


In [2]:
def filter_conllu(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            # print(line[0])
            cols = line.split("\t")
            if '.' in cols[0] or '-' in cols[0] or  '_	_	_	_	_	_	_' in line:
                continue
            else:
                outfile.write(line)

            # if '.' in line[0]:
            #         continue
            # else:
            #      outfile.write(line)

# Example usage
input_filename = '/home/preetamray-pg/g2g-transformer/data/UD_Marathi-UFAL/mr_ufal-ud-train.conllu'
output_filename = '/home/preetamray-pg/g2g-transformer/data/UD_Marathi-UFAL/mr_ufal-ud-train-clean.conllu'
filter_conllu(input_filename, output_filename)

#### Check exact match between prose and poetry of sentences of ramayana

In [1]:
import difflib

def get_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        sentences = content.split('\n')
        # You may need to adjust the splitting logic based on your specific file format.
        return [sentence.strip() for sentence in sentences if sentence.strip()]

def compare_sentences(sentence1, sentence2):
    words1 = sentence1.split()
    words2 = sentence2.split()
    matcher = difflib.SequenceMatcher(None, words1, words2)
    return matcher.ratio()  # Returns a similarity ratio between 0 and 1.

def main():
    file1_path = '/home/preetamray-pg/g2g-transformer/data/ramayana/prose.txt'
    file2_path = '/home/preetamray-pg/g2g-transformer/data/ramayana/poetry.txt'

    sentences1 = get_sentences(file1_path)
    sentences2 = get_sentences(file2_path)

    for i, (sentence1, sentence2) in enumerate(zip(sentences1, sentences2), 1):
        similarity_ratio = compare_sentences(sentence1, sentence2)
        with open("prose_poetry_similarity.txt", "a", encoding='UTF-8') as f:
            f.writelines(f"Comparison for sentence {i}: {similarity_ratio}")
            f.write("\n")

if __name__ == "__main__":
    main()


#### Extract sentences from a conllu file and write to another file

In [1]:
from conllu import parse
import random

def extract_and_remove_sentences(input_file_path, output_file_path, num_sentences):
    with open(input_file_path, 'r', encoding='utf-8') as input_file:
        data = input_file.read()

    sentences = parse(data)
    
    # Ensure the number of sentences to extract is not greater than the total number of sentences
    num_sentences = min(num_sentences, len(sentences))

    # Randomly select sentences to extract
    selected_sentences = random.sample(sentences, num_sentences)

    # Write extracted sentences to a new file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for sentence in selected_sentences:
            output_file.write(sentence.serialize())

    # Remove extracted sentences from the original file
    remaining_sentences = [s for s in sentences if s not in selected_sentences]

    # Write the remaining sentences back to the original file
    with open(input_file_path, 'w', encoding='utf-8') as input_file:
        for sentence in remaining_sentences:
            input_file.write(sentence.serialize())

# Example usage:
input_file_path = '/home/preetamray-pg/g2g-transformer/data/UD_Sanskrit-Vedic/sa_vedic-ud-train.conllu'
output_file_path = '/home/preetamray-pg/g2g-transformer/data/UD_Sanskrit-Vedic/sa_vedic-ud-dev.conllu'
num_sentences_to_extract = 500

extract_and_remove_sentences(input_file_path, output_file_path, num_sentences_to_extract)